# Simple Simulation Analysis

In this notebook I will analyse the output of a simple simulation. First, I load the output data of the two stations.

In [13]:
%matplotlib inline

import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from e_sim.sim_components import Simulator
from e_sim.utils import experiment_runner

## Run simulations

Run the simulation for several settings for the batch sizes and rates

In [14]:
# Common settings of the simulation
sim_time = 1000

# Settings dict
settings = {}
settings['demand_rate'] = [1]
settings['repair_rate'] = [0.5]
settings['Q_service'] = [1,2]
settings['Q_repair'] = [1,2,3,4]
settings['S_depot'] = [2]
settings['S_warehouse'] = [2]
settings['init_stock_depot'] = [4]
settings['init_stock_warehouse'] = [2, 8]

# Run all combinations of experiments
sim_dfs = experiment_runner(settings, sim_time)

ValueError: Index contains duplicate entries, cannot reshape

## Visualisation of stock over time

The stock levels of the repairable and servicable units are recorded over time. Let's check these.

In [0]:
# Get a feeling for the data
sim_dfs.head()

In [0]:
# Get all relevant inventory level variables
# Create long format
inventory_data = sim_dfs.melt(value_vars=['service_stock_depot', 'service_stock_position', 'service_back_orders', 'repair_stock_depot', 'service_stock_warehouse', 'repair_stock_warehouse'], 
                              id_vars=['time', 'settings'],
                              value_name='inventory_level', var_name='inventory_type'
                              )

# Plot lines over time 
g = sns.FacetGrid(inventory_data[inventory_data.time >= 0], col='settings', col_wrap=1, hue='inventory_type', aspect=3, sharey=False)

# Plot lines
g.map(plt.plot, 'time', 'inventory_level').add_legend().set_axis_labels('Time', 'Inventory level')

## Compute the average cycle cost

In this simple model, we assume that sending a batch from the warehouse to the depot has a cost `c_d` and sending a batch of repairable units from the depot to the warehouse costs `c_r`. In general we have that `c_r` > `c_d`, since the reverse flow is typically less efficient. Besides, set-up costs for orders, a holding costs of `h_d` per unit per unit of time for stock of servicable units at the depot has to be paid. In case of a shortage, a backlogging cost of `b_d` per unit per unit of time is incurred. For now, I do not assume any holding cost for the repairable units.

In [0]:
# Transform data types to float
sim_dfs.costs = sim_dfs.costs.astype(float)

# Total avg cost of simulation
sim_dfs.groupby(list(settings.keys())).agg({'costs':'mean'})

### Average inventory levels per cycle

In [0]:
sim_dfs.dtypes